In [16]:
import pprint
""" Import the GreenTranslator API wrapper. """
from greentranslator.api import GreenTranslator

""" Create an instance. """
translator = GreenTranslator ()

In [17]:
""" Query the exposure service. TODO: exercise other exposure endpoints. """
exposure = translator.\
    get_exposures().\
    get_exposure_by_area (exposure_type = 'pm25',
                        latitude      = '',
                        longitude     = '',
                        radius        = '0')
pprint.pprint (exposure, indent=2)

[ {'latitude': '35.7795897', 'longitude': '-78.6381787'},
  {'latitude': '35.9131996', 'longitude': '-79.0558445'},
  {'latitude': '35.9940329', 'longitude': '-78.898619'}]


In [18]:
""" Get drugs by condition. TODO: expand to specify a variety of related pheonotypes as parameters. """
drugs = translator.\
    get_medbiochem().\
    get_drugs_by_disease ("asthma")
pprint.pprint (drugs, indent=2)

[ 'Theophylline',
  'Beclomethasone',
  'Montelukast',
  'Fluticasone Propionate',
  'Dyphylline',
  'Hydrocortisone',
  'Salmeterol',
  'Formoterol',
  'Chlorpheniramine',
  'Aminophylline',
  'Vitamin E',
  'Flunisolide',
  'Nedocromil',
  'Zileuton',
  'Mometasone',
  'Enprofylline',
  'Isoproterenol',
  'Omalizumab',
  'Isoetharine',
  'Zafirlukast',
  'Cinalukast',
  'Terbutaline']


In [19]:
""" Get genes and pathways assoicated with a set of conditions. """
conditions = [ 'd001249', 'd003371', 'd001249' ]
genes_paths = translator.\
    get_medbiochem().\
    get_genes_pathways_by_disease (diseases = conditions)
pprint.pprint (genes_paths[:3], indent=2)

[ { 'is_human': True,
    'kegg_path': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa00010',
    'path_name': 'path:hsa00010 Glycolysis / Gluconeogenesis - Homo sapiens '
                 '(human)',
    'uniprot_gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/ALDH2'},
  { 'is_human': True,
    'kegg_path': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa00010',
    'path_name': 'path:hsa00010 Glycolysis / Gluconeogenesis - Homo sapiens '
                 '(human)',
    'uniprot_gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/ALDH2'},
  { 'is_human': True,
    'kegg_path': 'http://chem2bio2rdf.org/kegg/resource/kegg_pathway/hsa00010',
    'path_name': 'path:hsa00010 Glycolysis / Gluconeogenesis - Homo sapiens '
                 '(human)',
    'uniprot_gene': 'http://chem2bio2rdf.org/uniprot/resource/gene/ALDH2'}]


In [20]:
""" Get conditions associated with an environmental stressor agent. """
exposures = translator.\
    get_medbiochem().\
    get_exposure_conditions (stressorAgentIDs = [ 'D052638' ])
pprint.pprint (exposures[:5], indent=2)

[ { 'conditionID': 'http://bio2rdf.org/mesh:D001247',
    'stressorAgentName': 'Particulate Matter'},
  { 'conditionID': 'http://bio2rdf.org/mesh:D005317',
    'stressorAgentName': 'Particulate Matter'},
  { 'conditionID': 'http://bio2rdf.org/mesh:D011297',
    'stressorAgentName': 'Particulate Matter'},
  { 'conditionID': 'http://bio2rdf.org/mesh:D011297',
    'stressorAgentName': 'Particulate Matter'},
  { 'conditionID': 'http://bio2rdf.org/mesh:D001724',
    'stressorAgentName': 'Particulate Matter'}]
